# Module 09: Design and implement a replication strategy for Azure Cosmos DB SQL API

In [1]:
public class Product
{
	public string id { get; set; }
	public string categoryId { get; set; }
	public string categoryName { get; set; }
	public string sku { get; set; }
	public string name { get; set; }
	public string description { get; set; }
	public double price { get; set; }
}

## Configure replication and manage failovers in Azure Cosmos DB

### Understand replication

A replica set is a group of replicas that can dynamically grow and shrink to meet the needs of the database platform.

Each replica set will have other geographically distant replica sets that manage the same partition keys if data is distributed globally. These replica sets can then forward data to other replica sets in different regions to create replica copies of the data.

![image](https://docs.microsoft.com/en-us/learn/wwl-data-ai/configure-replication-manage-failovers-azure-cosmos-db/media/2-replica-sets.png)

An Azure Cosmos DB account replicates data within a region (local distribution) among different replica sets servicing various partition key values. Replica sets that manage the same partition key value are referred to as a partition set as they will forward data between each other (global distribution).

![image](https://docs.microsoft.com/en-us/learn/wwl-data-ai/configure-replication-manage-failovers-azure-cosmos-db/media/2-partition-sets.png)

### Distribute data across regions

Configuring global distribution in Azure Cosmos DB is a turnkey operation that is performed when an account is created or afterward.

Configuring geo-redundancy for a new account:

![image](https://docs.microsoft.com/en-us/learn/wwl-data-ai/configure-replication-manage-failovers-azure-cosmos-db/media/3-geo-redundancy.png)

Configuring geo-redundancy for an existing account:

![image](https://docs.microsoft.com/en-us/learn/wwl-data-ai/configure-replication-manage-failovers-azure-cosmos-db/media/3-replication-map.png)

The cost of distributing data globally is **RU/s x # of regions**.

For example, consider a solution that uses approximately 1,000 RU/s per hour; data is written to one Azure region and replicated to five more regions. The formula for this would be:

```bash
1,000 x (1+5) = 6,000
```

The account would be billed for **6,000 RU/s** used at a per-hour rate.

### Azure Cosmos DB failovers

An automatic failover plan can transfer the write region to one of the read regions in the case of an outage.

Define automatic failover policies:

![image](https://docs.microsoft.com/en-us/learn/wwl-data-ai/configure-replication-manage-failovers-azure-cosmos-db/media/5-automatic-failover.png)

Perform manual failovers:

![image](https://docs.microsoft.com/en-us/learn/wwl-data-ai/configure-replication-manage-failovers-azure-cosmos-db/media/6-manual-failover.png)

### Configure SDK region

Use the ApplicationRegion or ApplicationPreferredRegions properties to configure preferred regions.

In [ ]:
// Setting an application region

using Microsoft.Azure.Cosmos;

CosmosClientOptions options = new () { ApplicationRegion = Regions.UKSouth }; 
CosmosClient client = new (connectionString, options);

In [ ]:
//  Or using the CosmosClientBuilder class

using Microsoft.Azure.Cosmos.Fluent;

CosmosClient client = new CosmosClientBuilder(connectionString) 
    .WithApplicationRegion(Regions.UKSouth) 
    .Build();

In [ ]:
// Setting a list of preferred application regions

List<string> regions = new() { "East Asia", "South Africa North", "West US" }; 
CosmosClientOptions options = new () { ApplicationPreferredRegions = regions }; 
CosmosClient client = new (connectionString, options);

In [ ]:
//  Or using the CosmosClientBuilder class

CosmosClient client = new CosmosClientBuilder(connectionString) 
  .WithApplicationPreferredRegions( new List<string> 
    { 
        Regions.EastAsia, 
        Regions.SouthAfricaNorth, 
        Regions.WestUS 
    } ) .Build();

## Use consistency models in Azure Cosmos DB SQL API

### Understand consistency models

In a distributed database system, tradeoffs are often made between highly consistent data with extended latency and speedy data operations that may not be consistent immediately.

![image](https://docs.microsoft.com/en-us/learn/wwl-data-ai/use-consistency-models-azure-cosmos-db-sql-api/media/2-sliding-scale.png)

Each of the five consistency levels is well-defined with clear tradeoffs when compared with each other:

| **Consistency Level** | **Description** |
| ---: | --- |
| **Strong** | Linear consistency. Data is replicated and committed in all configured regions before acknowledged as committed and visible to all clients. |
| **Bounded Staleness** | Reads lag behind writes by a configured threshold in time or items. |
| **Session** | Within a specific session (SDK instance), users can read their own writes. |
| **Consistent Prefix** | Reads may lag behind writes, but reads will never appear out of order. |
| **Eventual** | Reads will eventually be consistent with writes. |

### Configure default consistency model in the portal

In the Azure portal, the Default consistency pane is used to configure a new default consistency level for the entire account.

![image](https://docs.microsoft.com/en-us/learn/wwl-data-ai/use-consistency-models-azure-cosmos-db-sql-api/media/3-default-consistency.png)

### Change consistency model with the SDK

Using the **ItemRequestOptions** class, you can relax the current default consistency level to a weaker one.

In [ ]:
// Set the item request Consistency Level option
ItemRequestOptions options = new() 
{ 
    ConsistencyLevel = ConsistencyLevel.Eventual 
};

// Assign the option to the create item operation
var item = new Product 
{ 
    id = $"{Guid.NewGuid()}", 
    categoryId = $"{Guid.NewGuid()}", 
    name = "Reflective Handlebar"
}; 

CosmosClient client = new (connectionString);
Database database = await client.CreateDatabaseIfNotExistsAsync("cosmicworks");
Container container = await database.CreateContainerAsync("cosmicworks", "/categoryId", 400 );

await container.CreateItemAsync<Product>(item, requestOptions: options);

### Use session tokens

Using the .NET SDK classes, the session token can be manually extracted and passed back to the Azure Cosmos DB resource.

In [ ]:
// Create an item with an Item Response
var id = $"{Guid.NewGuid()}";
var categoryId = $"{Guid.NewGuid()}";
var item = new Product 
{ 
    id = id, 
    categoryId = categoryId, 
    name = "Reflective Handlebar 2"
}; 
ItemResponse<Product> response = await container.CreateItemAsync<Product>(item); 

// Get the session token from the item response
string token = response.Headers.Session;

// Set the item request option session token to the previous token
ItemRequestOptions options = new() 
{ 
    SessionToken = token 
}; 

// Use the token on the new request
ItemResponse<Product> readResponse = await container.ReadItemAsync<Product>(
    id, new PartitionKey(categoryId), requestOptions: options);

Console.WriteLine($"Session token: {token}");

## Configure multi-region write in Azure Cosmos DB SQL API

### Understand multi-region write

With Azure Cosmos DB, every region supports both writes and reads. 

![image](https://docs.microsoft.com/en-us/learn/wwl-data-ai/configure-multi-region-write-azure-cosmos-db-sql-api/media/2-enable-multi-write.png)

### Understand conflict resolution policies

Azure Cosmos DB’s multi-region write feature has automatic conflict management built in. The default policy is known as Last Write Wins that uses the _ts property by default.

Replace the default _ts property configuring any numeric property as a conflict resolution path on the .NET SDK

In [ ]:
Database database = client.GetDatabase("cosmicworks"); 

// Define a default custom conflict resolution path as /metadata/sortableTimestamp.
ContainerProperties properties = new("products", "/categoryId") 
{ 
    ConflictResolutionPolicy = new ConflictResolutionPolicy() 
    { 
        Mode = ConflictResolutionMode.LastWriterWins, 
        ResolutionPath = "/sortableTimestamp"  // "/metadata/sortableTimestamp"  will return Invalid path '\/metadata\/sortableTimestamp' for last writer wins conflict resolution
    } 
}; 

// Note: You can only set a conflict resolution policy on newly created containers.
//await container.DeleteContainerAsync();
container = await database.CreateContainerIfNotExistsAsync(properties);

## Demo teardown

In [ ]:
CosmosClient client = new (connectionString);
Database database = client.GetDatabase("cosmicworks");

await database.DeleteAsync();